# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [9]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report , accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
import pickle
pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eppmi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eppmi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# load data from database

conn = sqlite3.connect(r'E:\Dropbox\Pessoal\Python\Udacity\Disaster-Response-Pipelines\databases\Disaster.db')


df = pd.read_sql('SELECT * FROM Disaster', con = conn)


df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.0,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000,26042.000000
mean,15148.289071,0.773558,0.170455,0.004531,0.414215,0.079525,0.049919,0.027763,0.018009,0.032793,0.0,0.063590,0.111589,0.088127,0.015321,0.023078,0.011443,0.033369,0.045542,0.131518,0.065087,0.045811,0.050879,0.020313,0.006067,0.010790,0.004570,0.011750,0.043967,0.278435,0.082405,0.093426,0.010752,0.093733,0.019968,0.052300,0.193303
std,8783.013166,0.435620,0.376040,0.067162,0.492596,0.270562,0.217783,0.164296,0.132988,0.178098,0.0,0.244025,0.314866,0.283485,0.122830,0.150154,0.106360,0.179602,0.208493,0.337973,0.246684,0.209078,0.219756,0.141073,0.077657,0.103316,0.067445,0.107762,0.205027,0.448237,0.274987,0.291034,0.103134,0.291463,0.139892,0.222636,0.394896
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7414.250000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15572.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22812.750000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30111.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
#Remove child alone as it has all zeros only
df = df.drop(['child_alone'],axis=1)

In [12]:
# Thr related column has more than a 2 categories
df.related.value_counts()

1    19765
0     6087
2      190
Name: related, dtype: int64

In [13]:
#Dropping the related rows with value  = 2
df = df[df['related'] != 2]

In [14]:

X = df['message']
Y = df.loc[:,'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    tokens = word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()).replace("  ",""))
    words = [w for w in tokens if w not in stopwords.words("english")]
    

    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    # Reduce words to their stems
    
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in stemmed]
    
    return lemmed 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer())
            ]))
            
        ])),

        ('clf', MultiOutputClassifier(KNeighborsClassifier(n_jobs=-1)))
    ])




### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)


In [20]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer()),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [21]:
y_pred = pipeline.predict(X_test)

In [22]:
for n, col in enumerate(y_test.columns):
    print(col)
    print(classification_report(y_test[col], y_pred[:,n]))

related
              precision    recall  f1-score   support

           0       0.34      0.11      0.17      1502
           1       0.78      0.94      0.85      4961

    accuracy                           0.74      6463
   macro avg       0.56      0.52      0.51      6463
weighted avg       0.68      0.74      0.69      6463

request
              precision    recall  f1-score   support

           0       0.84      0.95      0.89      5387
           1       0.32      0.12      0.18      1076

    accuracy                           0.81      6463
   macro avg       0.58      0.54      0.54      6463
weighted avg       0.76      0.81      0.77      6463

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6425
           1       0.00      0.00      0.00        38

    accuracy                           0.99      6463
   macro avg       0.50      0.50      0.50      6463
weighted avg       0.99      0.99      0.99      646

C:\Users\eppmi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



other_aid
              precision    recall  f1-score   support

           0       0.87      0.98      0.92      5602
           1       0.11      0.01      0.02       861

    accuracy                           0.85      6463
   macro avg       0.49      0.50      0.47      6463
weighted avg       0.77      0.85      0.80      6463

infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      6035
           1       0.00      0.00      0.00       428

    accuracy                           0.93      6463
   macro avg       0.47      0.50      0.48      6463
weighted avg       0.87      0.93      0.90      6463

transport
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      6155
           1       0.00      0.00      0.00       308

    accuracy                           0.95      6463
   macro avg       0.48      0.50      0.49      6463
weighted avg       0.91      0

In [9]:
# inspired by https://github.com/dabln/dsnd-DisasterResponsePipeline/blob/master/notebooks_code_development%20/ML%20Pipeline.ipynb
def report(y_test,y_pred, average = 'weighted', score=f1_score):
    report = {}
    for n, col in enumerate(y_test.columns):
        output = classification_report(y_test[col], y_pred[:,n], output_dict=True)
        report[col] = {}
        for i in output:   
            if i == 'accuracy':
                break
            report[col]['f1_' + i] = output[i]['f1-score']
            report[col]['precision_' + i] = output[i]['precision']
            report[col]['recall_' + i] = output[i]['recall']

    report_df = pd.DataFrame(report).transpose()
    report_df = report_df[report_df.columns.sort_values()]
    report_df_mean = report_df.mean()
    
    print("Table for each column:")
    print (report_df)
    print('\n')
    print('mean of results:')
    print(report_df_mean)
    print('\n')
    print('{} ({}): {}'.format(score, average, score(y_test, y_pred, average=average)))
    

In [14]:
report(y_test,y_pred)

Table for each column:
                            f1_0      f1_1  precision_0  precision_1  \
related                 0.268452  0.801023     0.319277     0.767780   
request                 0.868812  0.264691     0.854709     0.291667   
offer                   0.997984  0.000000     0.995977     0.000000   
aid_related             0.613161  0.394523     0.598071     0.410744   
medical_help            0.960347  0.000000     0.924149     0.000000   
medical_products        0.976726  0.000000     0.954658     0.000000   
search_and_rescue       0.985321  0.000000     0.971216     0.000000   
security                0.990235  0.000000     0.980659     0.000000   
military                0.981964  0.000000     0.964568     0.000000   
water                   0.969958  0.000000     0.942251     0.000000   
food                    0.941408  0.011065     0.892618     0.142857   
shelter                 0.956381  0.010989     0.917119     0.375000   
clothing                0.993459  0.00000

## 6 - 9. Improve your model , test your model, try improving your model further, Export your model as a pickle file. 

### I joined the parts 6 through 9. For this i made a class called Pipeline_scorer that can do the training and predict of the data and then give the requested results.
### For time purpouses i will keep the models simple.  

In [11]:
from Pipeline_scorer import Pipeline_Scorer

In [11]:
metrics       = ['minkowski','euclidean','manhattan'] 
weights       = ['uniform','distance'] #10.0**np.arange(-5,4)
numNeighbors  = np.arange(5,10)
parameters_grid    = dict(clf__estimator__metric=metrics,clf__estimator__weights=weights,clf__estimator__n_neighbors=numNeighbors)
pipe1 = Pipeline_Scorer(parameters_grid=parameters_grid,classifier=KNeighborsClassifier())

In [12]:
pipe1.pipeline(X_train, y_train, X_test,y_test,jobs=-1)

{'clf__estimator__metric': 'minkowski', 'clf__estimator__n_neighbors': 9, 'clf__estimator__weights': 'distance'}


In [13]:
pipe1.report()

C:\Users\eppmi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Table for each column:
                            f1_0      f1_1  precision_0  precision_1  \
related                 0.138075  0.850372     0.340206     0.770864   
request                 0.896528  0.189891     0.841881     0.386111   
offer                   0.997829  0.000000     0.995668     0.000000   
aid_related             0.677293  0.284292     0.585598     0.435545   
medical_help            0.957497  0.000000     0.918601     0.000000   
medical_products        0.975347  0.000000     0.952175     0.000000   
search_and_rescue       0.987625  0.000000     0.975553     0.000000   
security                0.990787  0.000000     0.981894     0.000000   
military                0.981964  0.000000     0.964568     0.000000   
water                   0.968725  0.000000     0.939929     0.000000   
food                    0.939797  0.000000     0.887117     0.000000   
shelter                 0.949533  0.000000     0.904334     0.000000   
clothing                0.992046  0.00000

In [15]:

pickle.dump(pipe1, open('cv_KNeighborsClassifier', "wb"))

In [18]:
learning_rate = [0.01,0.1] 
n_estimators = [50] #10.0**np.arange(-5,4)
criterion = ['friedman_mse', 'mse']
parameters_grid    = dict(clf__estimator__learning_rate=learning_rate,
                          clf__estimator__n_estimators=n_estimators,clf__estimator__criterion=criterion)
pipe2 = Pipeline_Scorer(parameters_grid=parameters_grid,classifier=GradientBoostingClassifier())

In [19]:
pipe2.pipeline(X_train, y_train, X_test,y_test,jobs=4)

{'clf__estimator__criterion': 'friedman_mse', 'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 50}


In [20]:
pipe2.report()

C:\Users\eppmi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Table for each column:
                            f1_0      f1_1  precision_0  precision_1  \
related                 0.005358  0.870113     0.400000     0.770804   
request                 0.903380  0.064463     0.830248     0.438202   
offer                   0.995101  0.030769     0.994560     0.034483   
aid_related             0.735367  0.040787     0.587169     0.478992   
medical_help            0.957833  0.000000     0.919078     0.000000   
medical_products        0.971597  0.000000     0.944762     0.000000   
search_and_rescue       0.979392  0.000000     0.968608     0.000000   
security                0.987381  0.035928     0.983604     0.050847   
military                0.982362  0.008850     0.968934     0.040000   
water                   0.966659  0.004773     0.935614     0.500000   
food                    0.938132  0.002649     0.883746     0.333333   
shelter                 0.951067  0.000000     0.906700     0.000000   
clothing                0.986748  0.02312

In [21]:
pickle.dump(pipe2, open('cv_GradientBoostingClassifier.pkl', "wb"))

In [22]:
max_features =["sqrt", "log2" ]
n_estimators = [50] 
criterion = ['gine', 'entropy']
parameters_grid    = dict(clf__estimator__max_features=max_features,
                          clf__estimator__n_estimators=n_estimators,clf__estimator__criterion=criterion)
pipe3 = Pipeline_Scorer(parameters_grid=parameters_grid,classifier=RandomForestClassifier())

In [23]:
pipe3.pipeline(X_train, y_train, X_test,y_test,jobs=4)

{'clf__estimator__criterion': 'entropy', 'clf__estimator__max_features': 'sqrt', 'clf__estimator__n_estimators': 50}


In [24]:
pipe3.report()

C:\Users\eppmi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Table for each column:
                            f1_0      f1_1  precision_0  precision_1  \
related                 0.075077  0.867003     0.429577     0.775036   
request                 0.905837  0.072208     0.831378     0.614286   
offer                   0.997207  0.000000     0.994430     0.000000   
aid_related             0.704567  0.240817     0.594056     0.461376   
medical_help            0.957581  0.000000     0.919040     0.000000   
medical_products        0.971597  0.000000     0.944762     0.000000   
search_and_rescue       0.984124  0.000000     0.968895     0.000000   
security                0.991574  0.000000     0.983289     0.000000   
military                0.984122  0.009804     0.969040     0.333333   
water                   0.966664  0.000000     0.935479     0.000000   
food                    0.938188  0.010526     0.884121     0.500000   
shelter                 0.951059  0.003306     0.906826     0.500000   
clothing                0.991023  0.00000

In [25]:
pickle.dump(pipe3, open('cv_RandomForestClassifier.pkl', "wb"))

In [12]:
learning_rate =[0.5,1 ]
n_estimators = [50,100] 
algorithm = ['SAMME', 'SAMME.R']
parameters_grid    = dict(clf__estimator__learning_rate= learning_rate,
                          clf__estimator__n_estimators=n_estimators,clf__estimator__algorithm=algorithm)
pipe4 = Pipeline_Scorer(parameters_grid=parameters_grid,classifier=AdaBoostClassifier())

In [13]:
pipe4.pipeline(X_train, y_train, X_test,y_test,jobs=-1)

{'clf__estimator__algorithm': 'SAMME.R', 'clf__estimator__learning_rate': 1, 'clf__estimator__n_estimators': 50}


In [14]:
pipe4.pipeline(X_train, y_train, X_test,y_test,jobs=4)

{'clf__estimator__algorithm': 'SAMME.R', 'clf__estimator__learning_rate': 1, 'clf__estimator__n_estimators': 50}


In [15]:
pipe4.report()

Table for each column:
                            f1_0      f1_1  precision_0  precision_1  \
related                 0.081194  0.863212     0.400000     0.771651   
request                 0.900862  0.207376     0.845450     0.435673   
offer                   0.997674  0.000000     0.995512     0.000000   
aid_related             0.716086  0.205173     0.596780     0.465955   
medical_help            0.958330  0.003854     0.921563     0.083333   
medical_products        0.974615  0.000000     0.951223     0.000000   
search_and_rescue       0.986750  0.011696     0.974299     0.250000   
security                0.991102  0.000000     0.982513     0.000000   
military                0.981643  0.000000     0.966341     0.000000   
water                   0.967242  0.000000     0.937432     0.000000   
food                    0.940202  0.008163     0.890875     0.100000   
shelter                 0.952751  0.006814     0.912166     0.105263   
clothing                0.991727  0.07017

In [16]:
pickle.dump(pipe4, open('cv_Adabosst.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [8]:
 pipeline = Pipeline([
            ('features', FeatureUnion([

                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer()),
                    ('transformer', TfidfTransformer())
                ]))
            ])),
            ('clf', MultiOutputClassifier(AdaBoostClassifier(algorithm = 'SAMME.R', learning_rate = 1, n_estimators = 50)))
        ])

In [18]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer()),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_jobs=-1)))])

In [20]:
pickle.dump(pipeline, open('cv_Adabosst.pkl', "wb"))